In [ ]:

! pip install segmentation_models_pytorch

import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageOps
import cv2
import random
import glob

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.distributed import DistributedSampler

import segmentation_models_pytorch as smp
from segmentation_models_pytorch.utils.losses import JaccardLoss
from segmentation_models_pytorch.utils.metrics import IoU

import warnings
import networkx as nx
import os
from sklearn.metrics import precision_score, recall_score, f1_score
#warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np

In [ ]:
def calculate_metric(model, num, augs=None, threshold=0.501):
        img_paths = '/images/' + num + '.tif'
        mask_paths = '/masks/mask' + num + '.tif'

        iou = IoU(threshold=threshold)
        scores = {'IoU': [], 'P': [], 'R': [], 'f1': []}
        
        print(img_paths)
        img = np.array(Image.open(img_paths))
        mask = np.array(Image.open(mask_paths))
        
        if augs:
            trans = augs(image=img, mask=mask)
            img, mask = trans['image'], trans['mask']
        
        mask[mask > 0] = 1
        pred = model(img)

        pred[pred > threshold] = 1
        pred[pred != 1] = 0
        
        scores['IoU'].append(iou(pred, mask).item())

        mask, pred = mask.numpy().flatten(), pred.flatten()
        scores['P'].append(precision_score(mask, pred))
        scores['R'].append(recall_score(mask, pred))
        scores['f1'].append(f1_score(mask, pred))
        return scores


Predictor = SimplePredictor(model.cuda(), device=device, sample_size=(1024,1024), pad_size=64)

for i in range(1,16):
  print(i)
  scores = calculate_metric(Predictor, str(i), augs=val_augs, threshold=0.5)
  print(pd.DataFrame(pd.DataFrame(scores).mean()).T)

#scores = calculate_metric(Predictor, str(15), augs=val_augs, threshold=0.5)
#print(pd.DataFrame(pd.DataFrame(scores).mean()).T)

Old vesion

In [ ]:

def calculate_metrics(model, data_path, augs=None, threshold=0.501):
    img_paths = glob.glob(data_path+'/images/*.tif') # возвращение список (возможно, пустой) путей, соответствующих шаблону 
    mask_paths = glob.glob(data_path+'/masks/mask*.tif')
    img_paths.sort()
    mask_paths.sort()
    iou = IoU(threshold=threshold)
    scores = {'IoU': [], 'P': [], 'R': [], 'f1': []}
    for i in range(len(img_paths)):
        img = np.array(Image.open(img_paths[i]))
        mask = np.array(Image.open(mask_paths[i]))
        if augs:
            trans = augs(image=img, mask=mask)
            img, mask = trans['image'], trans['mask']
        
        mask[mask > 0] = 1
        pred = model(img)

        pred[pred > threshold] = 1
        pred[pred != 1] = 0
        
        scores['IoU'].append(iou(pred, mask).item())

        mask, pred = mask.numpy().flatten(), pred.flatten()
        scores['P'].append(precision_score(mask, pred))
        scores['R'].append(recall_score(mask, pred))
        scores['f1'].append(f1_score(mask, pred))

    return scores

model = ModelPl.load_from_checkpoint('/dataset/example2.ckpt')
Predictor = SimplePredictor(model.cuda(), device=device, sample_size=(1024,1024), pad_size=64)
scores = calculate_metrics(Predictor, '/dataset/test', augs=val_augs, threshold=0.5)
pd.DataFrame(pd.DataFrame(scores).mean()).T
scores